In [ ]:
from libs import images
import json
import os
from pathlib import Path
from libs.path_ex import find_folder_upwards
import glob
import shutil

In [ ]:
found = find_folder_upwards(Path.cwd(), "ml_moldic_pores")
if found:
    found = str(found)
    src_path = f"{found}/data/thin_sections"
    print("Found source path:", src_path)
    
    with open("static/imgs_sections/metadata.json") as f:
        metadata = json.load(f)
    
    for filename in metadata:
        in_file = f"{src_path}/{filename}"
        out_file = f"static/imgs_sections/{filename}"
        if os.path.exists(in_file):
            if not os.path.exists(out_file):
                print("Copying:", in_file, "->", out_file)
                shutil.copy2(in_file, out_file)
            else:
                print("Already exists, skipping:", out_file)
        else:
            print("File not found:", in_file)

In [ ]:
with open("static/imgs_sections/metadata.json") as f:
    metadata = json.load(f)

percs = ["12.5", "25", "50"]

for name in metadata.keys():
    for percentage in percs:
        if os.path.exists(f"static/imgs_sections/{percentage}/{name}"):
            print(f"Skipping static/imgs_sections/{name} at {percentage}%")
            continue
        print(f"Resizing static/imgs_sections/{name} to {percentage}%")
        images.resize_image_file(
                f"static/imgs_sections/{name}",
                f"static/imgs_sections/{percentage}/{name}",
                float(percentage),
            )

In [ ]:
folder = "static/imgs_sections/cuts/"

def make_cropped(cut_info, src_filename, dst_filename, path):
    source_file = os.path.join("static/imgs_sections", src_filename)
    target_file = os.path.join(path, dst_filename)
    if not os.path.exists(source_file):
        print("    Source file not found, skipping:", source_file)
        return False
    if os.path.exists(target_file):
        print("    Target file already exists, skipping:", target_file)
        return False
    images.crop_image_file(
        source_file,
        target_file,
        cut_info["x"],
        cut_info["y"],
        cut_info["width"],
        cut_info["height"],
    )
    return True

def is_cut_info_valid(cut_info):
    required_keys = ["x", "y", "width", "height"]
    return all(key in cut_info for key in required_keys)

for file in glob.glob(os.path.join(folder, "**", "cut-metadata.json"), recursive=True):
    print("Found:", file)
    # process file here
    with open(file, "r") as f:
        data = json.load(f)
    path = os.path.dirname(file)
    print("Path:", path)
    for key in data.keys():
        print("  Key:", key)
        obj = data[key]
        if is_cut_info_valid(obj):
            if make_cropped(obj, key, key, path):
                print("    Successfully cropped:", key)
        else:
            any_found = False
            for key2 in obj.keys():
                obj2 = obj[key2]
                dst_filename = key2
                if is_cut_info_valid(obj2):
                    if not dst_filename.endswith(('.jpg', '.jpeg', '.png')):
                        dst_filename += os.path.splitext(key)[1]
                    if make_cropped(obj2, key, dst_filename, path):
                        print("    Successfully cropped:", key)
                    any_found = True
                else:
                    print("    Missing cutting info:", key2)
            if not any_found:
                print("    Missing cutting info:", key)

In [ ]:
folder = "static/imgs_sections/cuts/"

for file in glob.glob(os.path.join(folder, "**", "cut-metadata.json"), recursive=True):
    print("Found:", file)
    # process file here
    with open(file, "r") as f:
        data = json.load(f)
    path = os.path.dirname(file)
    print("Path:", path)
    for filename in data.keys():
        print("  Key:", filename)
        source_file = os.path.join(path, filename)
        for percentage in percs:
            target_file = os.path.join(path, percentage, filename)
            if not os.path.exists(source_file):
                print("    Source file not found, skipping:", source_file)
                continue
            if os.path.exists(target_file):
                print("    Target file already exists, skipping:", target_file)
                continue
            os.makedirs(os.path.dirname(target_file), exist_ok=True)
            print("    Resizing to", percentage, ":", filename)
            images.resize_image_file(
                source_file,
                target_file,
                float(percentage),
            )